<a href="https://colab.research.google.com/github/amasl0/myKNN/blob/main/TicTacToeipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pickle
import random
import itertools
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import keras
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.utils import to_categorical

BOARD_SIZE = 3
BOARD_DIMENSIONS = (BOARD_SIZE, BOARD_SIZE)

CELL_EMPTY = 0
CELL_X = 1
CELL_O = 2

CELL_RANDOM_STRAT = 0
CELL_NETWORK_STRAT = 1

RESULT_X_WINS = 1
RESULT_O_WINS = -1
RESULT_DRAW = 0

PATH_GAMES = os.path.join( os.getcwd(), 'drive/MyDrive/TicTac' )

### Игровая доска состоит из матрицы 3 на 3, значения которых имеют 3 состояния:


> 0 - Пустая клетка

> 1 - Крестик

> 2 - Нолик

In [ ]:
## TicTacToe V2

class Board:
    def __init__ (self, new_board=None):
        if new_board is None:
            self.board = np.zeros(BOARD_DIMENSIONS, dtype=int)
        else:
            self.board = np.copy(new_board)
    
        self.history = []

    ## Провреяет, все ли значения одинаковы
    def check_equal_values (self, lst):
        g = itertools.groupby( lst )
        return next(g, True) and not next(g, False)

    def insert_value (self, value, position):
        self.board[position[0]][position[1]] = value

    ## Проверка на корректность позиции
    def is_valid_position ( self, position ):
        return True if self.board[position[0]][position[1]] == CELL_EMPTY else False

    ## Взять все координаты пустых полей
    def get_valid_positions (self):
        return np.argwhere(self.board == CELL_EMPTY)

    def get_winner (self):
        trans_board = np.transpose(np.copy(self.board))

        for ix in range(BOARD_SIZE):
            if self.check_equal_values( (self.board[ix][:BOARD_SIZE] )) and self.board[ix][BOARD_SIZE-1] != CELL_EMPTY:
                return self.board[ix][BOARD_SIZE-1] 

            if self.check_equal_values( (trans_board[ix][:BOARD_SIZE] )) and trans_board[ix][BOARD_SIZE-1] != CELL_EMPTY:
                return trans_board[ix][BOARD_SIZE-1] 
        

        if self.check_equal_values(np.diag(self.board)) and self.board[0][0] != CELL_EMPTY:
            return self.board[0][0]

        elif self.check_equal_values( np.diag( np.fliplr( self.board )) ) and self.board[0][-1] != CELL_EMPTY:
            return self.board[0][-1]
        
        return -1

    def is_gameover (self):
        winner = self.get_winner()
        
        ## Остались ли ходы?
        if winner == -1 and len(self.get_valid_positions()) == 0:
                return True, 0
        elif winner != -1:
            return True, winner
        return False, -1

    def draw_point(value):
        if value == 1:
            return 'X'
        elif value == 2:
            return 'O'
        else: 
            return ' '

    def draw_board( self ):

        print("\n") 

        for ix in range(len(self.board) - 1):
            print("\t     |     |") 
            print("\t  {}  |  {}  |  {}".format(self.board[ix][0], self.board[ix][1], self.board[ix][2])) 
            print('\t_____|_____|_____') 
        print("\t     |     |") 
        print("\t  {}  |  {}  |  {}".format(self.board[2][0], self.board[2][1], self.board[2][2])) 
        print("\t     |     |") 
        print("\n")

    def addHistory (self):
        self.history.append(np.copy(self.board))

class Player:
    def __init__ (self, rival):
        self.rival = rival 

    def move (self, board, position):
        if board.is_valid_position(position):
            board.insert_value( self.rival, position )
            board.addHistory()
            return True
        else:
            return False

class RandomPlayer(Player):
    def move (self, board):
        empties_fields = board.get_valid_positions()
        super().move( board, empties_fields[random.randrange(len(empties_fields))] )

class AiPlayer(Player):
    def __init__(self, rival, neural):
        self.neural = neural
        super().__init__(rival)

    def move (self, board):
        ## Получаем список доступных ходов
        empties_fields = board.get_valid_positions()
        
        if len(empties_fields) == 0:
            return False

        max_win_pos = 0.0
        win_position = (-1, -1)

        for probably_field in empties_fields:
            copy_board = np.copy( board.board )
            copy_board[probably_field[0]][probably_field[1]] = self.rival
            prob_win = self.neural.predict(copy_board, self.rival )

            if prob_win > max_win_pos:
                max_win_pos = prob_win
                win_position = probably_field
            
        super().move( board, win_position )

## для игры между игроками
def playgame_users ():
    playboard = Board()
    count_actions = 9
    player = 0

    while True:
        if count_actions % 2 == 0:
            player = Player(CELL_O)
        else:
            player = Player(CELL_X)
    
        print(f"Ходит игрок { 'X' if player.rival == 1 else 'O' }")
        playboard.draw_board()

        position = tuple(map(int, input().split(",")))

        if playboard.is_valid_position( position ):
            player.move( playboard, position )
            count_actions -= 1
            res = playboard.is_gameover()

            if res[0]:
                if res[1] == 1:
                    return RESULT_X_WINS
                elif res[1] == 2:
                    return RESULT_O_WINS
                elif res[1] == 0:
                    return RESULT_DRAW

def play_game_rand_bots (count_games=10000):

    all_games = {
        'X_wins' : [],
        'O_wins' : [],
        'draw' : []
    }

    for counter_games in tqdm( range( count_games ) ):
        board = Board()
        count_actions = 9
        player = RandomPlayer(CELL_X)

        while True:
            player = RandomPlayer(CELL_O) if count_actions % 2 == 0 else RandomPlayer(CELL_X)
            empties_fields = board.get_valid_positions()
            player.move(board)
            count_actions -= 1

            res = board.is_gameover()
            if res[0]: ## если игра окончена
                if res[1] == 1:
                   all_games["X_wins"].append(board.history) 
                elif res[1] == 2:
                    all_games["O_wins"].append(board.history) 
                elif res[1] == 0:
                    all_games["draw"].append(board.history) 
                break
    return all_games

def play_game_user_vs_ai(neural):
    playboard = Board()
    count_actions = 9
    player = 0

    while True:
        print(f"Ходит игрок { 'X' if count_actions % 2 == 1 else 'O' }")
        playboard.draw_board()
        if count_actions % 2 != 0:
            player = Player(CELL_X)
            position = tuple(map(int, input().split(",")))

            if playboard.is_valid_position( position ):
                player.move( playboard, position )
        else:
            player = AiPlayer(CELL_O, neural)
            player.move( playboard )
        
        count_actions -= 1
        res = playboard.is_gameover()

        if res[0]:
            if res[1] == 1:
                return RESULT_X_WINS
            elif res[1] == 2:
                return RESULT_O_WINS
            elif res[1] == 0:
                return RESULT_DRAW

### Заставим ботов играть против друг друга, записывая все ходы и состояния

In [25]:
res_games = play_game_rand_bots(10 ** 6)

100%|██████████| 10/10 [00:00<00:00, 945.02it/s]


In [3]:
# with open(os.path.join( PATH_GAMES,'all_games.pkl'), 'wb') as f:
#     pickle.dump(all_games, f)
  
with open(os.path.join( PATH_GAMES,'all_games.pkl'), 'rb') as f:
    all_games = pickle.load(f)

### Умея на руках данные о 100 000 партий можем закинуть их в нейронную сеть.

Перед этим следует сконструировать саму модель нейронной сети

In [4]:
class NeuralBot:

    def __init__(self, numberOfInputs, numberOfOutputs, epochs, batchSize):
        self.epochs = epochs
        self.batchSize = batchSize
        self.numberOfInputs = numberOfInputs
        self.numberOfOutputs = numberOfOutputs

        ## construct model
        self.model = Sequential()
        self.model.add(Dense(64, activation='relu', input_shape=(numberOfInputs, )))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(numberOfOutputs, activation='softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    def train(self, dataset):
        input = []
        output = []

        for key in dataset:
            for game in dataset[key]:
                for data in game:
                    input.append(data)

                    game_res = -1
                    if key == 'X_wins':
                        game_res = 1
                    elif key == 'O_wins':
                        game_res = 2
                    elif key == 'draw':
                        game_res = 0
                    output.append(game_res)

        X = np.array(input).reshape((-1, self.numberOfInputs))
        y = to_categorical(output, num_classes=3)
        # Train and test data split
        boundary = int(0.8 * len(X))
        X_train = X[:boundary]
        X_test = X[boundary:]
        y_train = y[:boundary]
        y_test = y[boundary:]

        self.history = self.model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=self.epochs, batch_size=self.batchSize)
        self.model.save(os.path.join( PATH_GAMES, 'model.h5' ))

    def predict(self, data, index):
        return self.model.predict(np.array(data).reshape(-1, self.numberOfInputs))[0][index]

In [5]:
neural = NeuralBot(9, 3, 1, 32)

In [6]:
neural.train(all_games)

190660/190660 [==============================] - 474s 2ms/step - loss: 0.5063 - accuracy: 0.7608 - val_loss: 1052.4542 - val_accuracy: 0.0906


## Поиграем с ботом

In [17]:
play_game_user_vs_ai(neural)

Ходит игрок X


	     |     |
	  0  |  0  |  0
	_____|_____|_____
	     |     |
	  0  |  0  |  0
	_____|_____|_____
	     |     |
	  0  |  0  |  0
	     |     |


1,1
Ходит игрок O


	     |     |
	  0  |  0  |  0
	_____|_____|_____
	     |     |
	  0  |  1  |  0
	_____|_____|_____
	     |     |
	  0  |  0  |  0
	     |     |


1/1 [==============================] - 0s 17ms/step
Ходит игрок X


	     |     |
	  0  |  0  |  0
	_____|_____|_____
	     |     |
	  0  |  1  |  0
	_____|_____|_____
	     |     |
	  0  |  0  |  2
	     |     |


0,2
Ходит игрок O


	     |     |
	  0  |  0  |  1
	_____|_____|_____
	     |     |
	  0  |  1  |  0
	_____|_____|_____
	     |     |
	  0  |  0  |  2
	     |     |


1/1 [==============================] - 0s 20ms/step
Ходит игрок X


	     |     |
	  0  |  0  |  1
	_____|_____|_____
	     |     |
	  0  |  1  |  0
	_____|_____|_____
	     |     |
	  2  |  0  |  2
	     |     |


2,1
Ходит игрок O


	     |     |
	  0  |  0  |  1
	_____|_____|_____
	  

1